## Comparison of PGKB and VEP genes

We focus exclusively on RS ID records, as for named alleles we always use PGKB genes and do not go through VEP.

First we look at the file of mismatches generated from the 2023-12 submission to get a quick idea of what we're dealing with.

In [1]:
import os
import csv
import re
from collections import Counter

In [2]:
def string_to_set(s):
    if s == 'set()':
        return set()
    return set(x for x in re.sub(r"{|}|'", '', s).split(', ') if x)

In [5]:
pgkb_vep_genes = []

# comparison file from 2023-12 release
filename = '/home/april/projects/opentargets/pharmgkb/vep-vs-pgkb/vep_vs_pgkb_genes.csv'

with open(filename) as f:
    reader = csv.reader(f)
    next(reader)  # skip header
    for row in reader:
        pgkb_vep_genes.append({
            'id': row[0],
            'pgkb': string_to_set(row[1]),
            'vep': string_to_set(row[2])
        })

In [6]:
# NB. includes only records where do not match
# See here: https://github.com/EBIvariation/opentargets-pharmgkb/issues/23
len(pgkb_vep_genes)

770

In [38]:
# Counting logic borrowed from CMAT paper

def get_f1_tp_fp_fn(pgkb_set, vep_set):
    """Returns f1-score and number of true positives, false positives and false negatives."""
    if len(pgkb_set) == 0 and len(vep_set) == 0:
        return 0, 0, 0, 0
    tp = len(vep_set & pgkb_set)
    fp = len(vep_set - pgkb_set)
    fn = len(pgkb_set - vep_set)
    return 2 * tp / (2 * tp + fp + fn), tp, fp, fn
    

def count_and_match(pgkb_set, vep_set, counts):
    """Return True if "match", False otherwise (mismatch == sets disjoint)"""
    pgkb_set = set(pgkb_set)
    vep_set = set(vep_set)

    # First check if either set is empty
    if not pgkb_set and vep_set:
        counts['pgkb_missing'] += 1
    elif pgkb_set and not vep_set:
        counts['vep_missing'] += 1
    elif not pgkb_set and not vep_set:
        counts['both_missing'] += 1

    # Both sets present, place in correct category
    else:
        _, true_pos, false_pos, false_neg = get_f1_tp_fp_fn(pgkb_set, vep_set)
        if false_pos and not false_neg:
            k = 'vep_superset'
        elif not false_pos and false_neg:
            k = 'vep_subset'
        elif not false_pos and not false_neg:
            k = 'exact_match'
        elif true_pos:
            k = 'divergent_match'
        else:
            k = 'mismatch'
        counts[k] += 1
        if k == 'mismatch':
            return False
    return True

In [8]:
counts = Counter()
mismatches = []

for record in pgkb_vep_genes:
    if not count_and_match(record['pgkb'], record['vep'], counts):
        mismatches.append(record)

In [9]:
# Missing counts: exact_match, both_missing
counts

Counter({'vep_superset': 140,
         'vep_subset': 273,
         'mismatch': 106,
         'vep_missing': 182,
         'pgkb_missing': 61,
         'divergent_match': 8})

In [ ]:
with open('/home/april/projects/opentargets/pharmgkb/vep-vs-pgkb/vep_vs_pgkb_mismatches.tsv', 'w+') as f:
    f.write('id\tpgkb\tvep\n')
    f.write('\n'.join('\t'.join([x['id'], ','.join(x['pgkb']), ','.join(x['vep'])]) for x in mismatches))

### Complete counts

From the above we observe that the mismatching genes seem to be in close proximity, so we want to also look at the variant locations. We're also missing some counts, particularly cases where both PGKB and VEP genes are missing, which would be useful to get.

Since we're running this again, might as well use the newest version of PGKB data though the differences will be minor (if anything).

In [10]:
from opentargets_pharmgkb.evidence_generation import explode_and_map_genes, get_functional_consequences, get_genotype_ids, ID_COL_NAME
from opentargets_pharmgkb.pandas_utils import read_tsv_to_df

import pandas as pd

In [11]:
data_dir = '/home/april/projects/opentargets/pharmgkb/vep-vs-pgkb'

In [ ]:
# Download new data (2024-03-05)
!cd {data_dir}

!wget -q https://api.pharmgkb.org/v1/download/file/data/clinicalAnnotations.zip
!wget -q https://api.pharmgkb.org/v1/download/file/data/variants.zip

!unzip -jq clinicalAnnotations.zip "*.tsv" -d {data_dir}
!unzip -jq variants.zip "*.tsv" -d {data_dir}

!rm clinicalAnnotations.zip variants.zip

In [13]:
# Adapted from evidence generation pipeline
clinical_annot_path = os.path.join(data_dir, 'clinical_annotations.tsv')
clinical_alleles_path = os.path.join(data_dir, 'clinical_ann_alleles.tsv')
variants_path = os.path.join(data_dir, 'variants.tsv')
fasta_path = '/home/april/projects/opentargets/pharmgkb/assembly/GCF_000001405.40_GRCh38.p14_genomic.fna'

clinical_annot_table = read_tsv_to_df(clinical_annot_path)
clinical_alleles_table = read_tsv_to_df(clinical_alleles_path)
variants_table = read_tsv_to_df(variants_path)

In [14]:
merged_with_variants_table = pd.merge(clinical_annot_table, variants_table, left_on='Variant/Haplotypes', right_on='Variant Name', how='left')
merged_with_alleles_table = pd.merge(merged_with_variants_table, clinical_alleles_table, on=ID_COL_NAME, how='left')

In [15]:
with_rs = merged_with_alleles_table[merged_with_alleles_table['Variant/Haplotypes'].str.startswith('rs')]

In [16]:
genotype_ids_table = get_genotype_ids(with_rs, fasta_path)

ERROR:opentargets_pharmgkb:Could not parse genotype (AGCCCACCC)12/(AGCCCACCC)12
ERROR:opentargets_pharmgkb:Could not parse genotype (CCGCGCCACTTGGCCTGCCTCCGTCCCG)2/(CCGCGCCACTTGGCCTGCCTCCGTCCCG)2
ERROR:opentargets_pharmgkb:Could not parse genotype (CCGCGCCACTTGGCCTGCCTCCGTCCCG)2/(CCGCGCCACTTGGCCTGCCTCCGTCCCG)3
ERROR:opentargets_pharmgkb:Could not parse genotype (CCGCGCCACTTGGCCTGCCTCCGTCCCG)3/(CCGCGCCACTTGGCCTGCCTCCGTCCCG)3
ERROR:opentargets_pharmgkb:Could not parse genotype (AGCCCACCC)12/(AGCCCACCC)12
ERROR:opentargets_pharmgkb:Could not parse genotype (CCGCGCCACTTGGCCTGCCTCCGTCCCG)2/(CCGCGCCACTTGGCCTGCCTCCGTCCCG)2
ERROR:opentargets_pharmgkb:Could not parse genotype (CCGCGCCACTTGGCCTGCCTCCGTCCCG)2/(CCGCGCCACTTGGCCTGCCTCCGTCCCG)3
ERROR:opentargets_pharmgkb:Could not parse genotype (CCGCGCCACTTGGCCTGCCTCCGTCCCG)3/(CCGCGCCACTTGGCCTGCCTCCGTCCCG)3
ERROR:opentargets_pharmgkb:Could not parse genotype (CCGCGCCACTTGGCCTGCCTCCGTCCCG)2/(CCGCGCCACTTGGCCTGCCTCCGTCCCG)2
ERROR:opentargets_pharmgkb:C

In [17]:
rs_consequences_table = get_functional_consequences(genotype_ids_table)

In [18]:
mapped_genes = explode_and_map_genes(rs_consequences_table)

INFO:cmat.consequence_prediction.common.biomart:Processing chunk of 741 records
INFO:cmat.consequence_prediction.common.biomart:Processing chunk of 337 records


In [32]:
# Re-group by ID column
genes_table = mapped_genes.groupby(by=ID_COL_NAME).aggregate(
    original_gene=('Gene', lambda x: x.iloc[0] if len(x) > 0 else None),
    variant_location=('Location', lambda x: x.iloc[0] if len(x) > 0 else None),
    all_pgkb_genes=('gene_from_pgkb', lambda x: set(x.dropna())),
    all_vep_genes=('overlapping_gene', lambda x: set(x.dropna()))
)

In [33]:
genes_table

,original_gene,variant_location,all_pgkb_genes,all_vep_genes
Clinical Annotation ID,,,,
1043858606,BDNF,NC_000011.10:27658369,{ENSG00000176697},{ENSG00000176697}
1043858615,GNB3,NC_000012.12:6845711,{ENSG00000111664},"{ENSG00000111665, ENSG00000111664}"
1043858632,TPH1,NC_000011.10:18026269,{ENSG00000129167},{ENSG00000129167}
1043858637,CEP68,NC_000002.12:65069664,{ENSG00000011523},{ENSG00000011523}
1043858642,FSIP1,NC_000015.10:39770852,{ENSG00000150667},{ENSG00000150667}
...,...,...,...,...
982046386,ADRA2C,NC_000004.12:3767577_3767588,{ENSG00000184160},{}
982047840,GCLC,NC_000006.12:53527310,{ENSG00000001084},{ENSG00000001084}
982047849,DRD3,NC_000003.12:114171968,{ENSG00000151577},{ENSG00000151577}


In [34]:
genes_table.to_csv(os.path.join(data_dir, 'all-pgkb-vep-genes.csv'))

In [62]:
complete_counts = Counter()
complete_mismatches = []

for record in genes_table.itertuples():
    if not count_and_match(record.all_pgkb_genes, record.all_vep_genes, complete_counts):
        complete_mismatches.append(record)

In [63]:
complete_counts

Counter({'exact_match': 3535,
         'vep_superset': 143,
         'vep_subset': 271,
         'both_missing': 196,
         'mismatch': 106,
         'vep_missing': 185,
         'pgkb_missing': 61,
         'divergent_match': 8})

In [64]:
# records without an original gene in PGKB
len(genes_table[genes_table['original_gene'].isna()])

245

In [65]:
# records with a gene we couldn't map to ensembl gene id
genes_table[~genes_table['original_gene'].isna() & (genes_table['all_pgkb_genes'].str.len() == 0)]

,original_gene,variant_location,all_pgkb_genes,all_vep_genes
Clinical Annotation ID,,,,
1183547855,UGT1A,NC_000002.12:233772999,{},"{ENSG00000242515, ENSG00000244122, ENSG0000024..."
1183614816,UGT1A,NC_000002.12:233772770,{},"{ENSG00000242515, ENSG00000244122, ENSG0000024..."
1183614825,UGT1A,NC_000002.12:233772770,{},"{ENSG00000242515, ENSG00000244122, ENSG0000024..."
1183614855,UGT1A,NC_000002.12:233772898,{},"{ENSG00000242515, ENSG00000244122, ENSG0000024..."
1183614860,UGT1A,NC_000002.12:233772898,{},"{ENSG00000242515, ENSG00000244122, ENSG0000024..."
1183698808,NPPA-AS1,NC_000001.11:11847591,{},{ENSG00000175206}
1445401413,UGT1A,NC_000002.12:233774704,{},{ENSG00000185038}
1445401420,UGT1A,NC_000002.12:233774704,{},{ENSG00000185038}
1448112614,RARS,NC_000005.10:168491921,{},{ENSG00000113643}


In [66]:
# confirm the record highlighted by OT has a gene
genes_table.loc['1447989678']

original_gene                                 DPYD
variant_location    NC_000001.11:97740411_97740418
all_pgkb_genes                   {ENSG00000188641}
all_vep_genes                                   {}
Name: 1447989678, dtype: object

In [77]:
def write_genes_df(df, outfile):
    # df is actually an iterable of named tuples, not a dataframe...
    with open(outfile, 'w+') as f:
        f.write('id\tlocation\toriginal\tpgkb\tvep\n')
        f.write('\n'.join('\t'.join([x.Index, x.variant_location, x.original_gene, ','.join(x.all_pgkb_genes), ','.join(x.all_vep_genes)])
                          for x in df))

In [78]:
# save the mismatches again, but with original gene & variant location
write_genes_df(complete_mismatches, os.path.join(data_dir, 'complete_vep_vs_pgkb_mismatches.tsv'))

In [81]:
# also save the unmapped genes
write_genes_df(genes_table[~genes_table['original_gene'].isna() & (genes_table['all_pgkb_genes'].str.len() == 0)].itertuples(),
               os.path.join(data_dir, 'unmapped_genes.tsv'))